# Initial test of data plotting
- Early stage build of a project. 
- This is a test at pulling images from a h5 file and displaying them.
- Future projects will use a modified version of this code. 

The data used is from Oak Ridge National Labratory<br>
**Big Data Analytics for Scanning Transmission Electron Microscopy Ptychography**

# Initilaization

In [ ]:
# System check
import sys
!conda install --yes --prefix {sys.prefix} numpy scipy matplotlib scikit-learn Ipython ipywidgets h5py
!{sys.executable} -m pip install -U --no-deps pyUSID==0.0.4
!{sys.executable} -m pip install -U --no-deps pycroscopy==0.60.1

In [ ]:
# Import Libraries
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import display, HTML
import ipywidgets as widgets
from sklearn.cluster import KMeans

sys.path.append('..')
import pyUSID as usid
import pycroscopy as px

# Make Notebook take up most of page width
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

# set up notebook to show plots within the notebook
%matplotlib notebook

# Load the dataset

In [ ]:
# The user will select the location of the data
h5_path = px.io_utils.file_dialog('*.h5', '4D STEM dataset formatted according to USID')
print('Working on:\n' + h5_path)
# Open the file
h5_file = h5py.File(h5_path, mode='r+')